#INTRODUCTION
This notebook introduces training and testing of a Convolution Neural Network (CNN) model using keras and Tensorflow. The model in question will later be used in a device running esp32 to guide a blind person by identifying and clarifying cars and people this avoiding possible collision.
As the model will be deployed in a resource constrained device, the notebook goes into detail on how edge impulse will be used to deploy the Tensorflow lite model which has been obtained from conversion of the original model to a tflite model.

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from google.colab import drive
import time
from datetime import datetime

t = datetime.now().strftime("%H:%M")
model_name = 'gray'

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Create a folder in Google Drive
data_path = '/content/drive/MyDrive/dataset'
os.makedirs(data_path, exist_ok=True)
print(f"Folder '{data_path}' created in Google Drive.")

save_directory = '/content/drive/MyDrive/models'
os.makedirs(save_directory, exist_ok=True)
save_path = os.path.join (save_directory, f"{model_name}.keras")


save_directory = '/content/drive/MyDrive/models'
os.makedirs(save_directory, exist_ok=True)
save_path_lite = os.path.join (save_directory, f"{model_name}.tflite")


Test_dir = '/content/drive/MyDrive/Test_img'
os.makedirs(Test_dir, exist_ok=True)
Test_folder= '/content/drive/MyDrive/Test_img/test'
os.makedirs(Test_folder, exist_ok=True)

Ard_dir = '/content/drive/MyDrive/Arduino'
os.makedirs(Ard_dir, exist_ok=True)
Lib_folder= '/content/drive/MyDrive/Arduino/Library'
os.makedirs(Lib_folder, exist_ok=True)

In [ ]:
DATADIR = data_path
CATEGORIES = ['person','car']

for category in CATEGORIES:
    path = os.path.join (DATADIR, category)
    for img in os.listdir (path):
        img_array = cv2.imread (os.path.join(path, img),cv2.IMREAD_GRAYSCALE)
        img_array = img_array/255
        plt.imshow (img_array, cmap = 'gray')
        plt.show ()
        break
    break

In [ ]:
IMG_SIZE = 50
new_array = cv2.resize (img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow (new_array, cmap = 'gray')
plt.show ()

In [ ]:
training_data = []

def create_training_data ():
    for category in CATEGORIES:
        path = os.path.join (DATADIR, category)
        class_num = CATEGORIES.index (category)
        for img in os.listdir (path):
            try:
                img_array = cv2.imread (os.path.join(path, img),cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize (img_array, (IMG_SIZE, IMG_SIZE))
                new_array = new_array/255
                training_data.append ([new_array, class_num])
            except Exception as e:
                pass

create_training_data ()

In [ ]:
import random
for i in range (5678):
    random.shuffle (training_data)

In [ ]:
from tensorflow.keras.utils import to_categorical
X = []
y = []
for features, label in training_data:
    X.append (features)
    y.append (label)
X = np.array (X).reshape (-1,IMG_SIZE,IMG_SIZE,1)
y = np.array (y)
count_zeros = np.count_nonzero(y == 1)
print (count_zeros)
y = to_categorical(y, num_classes=2)

In [ ]:
print (y.shape)
print (X.shape)

In [ ]:
print (len(training_data))
for sample in training_data[:10]:
   print (sample[1] )

In [ ]:
#saving training data
import pickle
save_directory ='/content/drive/MyDrive/Training_data'
os.makedirs(save_directory, exist_ok=True)
save_path_X = os.path.join (save_directory, f"X-{model_name}.pickle")
save_path_y = os.path.join (save_directory, f"y-{model_name}.pickle")

pickle_out = open (save_path_X, "wb")
pickle.dump (X, pickle_out)
pickle_out.close ()

pickle_out = open (save_path_y, "wb")
pickle.dump (y, pickle_out)
pickle_out.close ()

In [ ]:

#loading training data
import pickle
save_directory ='/content/drive/MyDrive/Training_data'
os.makedirs(save_directory, exist_ok=True)
save_path_X = os.path.join (save_directory, f"X-{model_name}.pickle")
save_path_y = os.path.join (save_directory, f"y-{model_name}.pickle")


pickle_in = open (save_path_X, "rb")
X = pickle.load (pickle_in)

pickle_in = open (save_path_y, "rb")
y = pickle.load (pickle_in)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

#Convolution Neural Network
Tensorflow and keras are used to train the model. The various parameters are adjusted until the model achieves an accuracy that can be used in real world scenario.

In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

l_r = 0.0001
Epochs = 15
opt = Adam(learning_rate = l_r)
batch_size = 32

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:],kernel_regularizer=regularizers.l1(0.01)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation("relu"))

model.add(Dense(2))
model.add(Activation ("softmax"))


model.compile(loss = "binary_crossentropy",
              optimizer = opt,
              metrics = ["accuracy"])

datagen.fit(X_train)

train_generator = datagen.flow(X_train, y_train, batch_size = batch_size)

#print(type(train_generator))

history = model.fit(train_generator, steps_per_epoch=len(X_train) // batch_size, epochs=Epochs, validation_data=(X_val, y_val))

#checkpoint = ModelCheckpoint(save_path,save_freq = 'epoch', monitor='val_loss', save_best_only=True, save_weights_only=False, mode='auto')
#history = model.fit(X,y, batch_size = 32, epochs = Epochs, verbose = 1, callbacks = [checkpoint])

In [ ]:
model.summary ()

In [ ]:
model.save(save_path)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.legend()
plt.show()

Image test

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model(save_path)

print(f"Model loaded successfully: {model}")

In [ ]:
directory_path = Test_dir
folder_name = Test_folder
test_folder_path = os.path.join(directory_path, folder_name)
IMG_SIZE = 50

if os.path.exists(test_folder_path):
    image_files = [f for f in os.listdir(test_folder_path) if os.path.isfile(os.path.join(test_folder_path, f))]
    for image_file in image_files:
        image_path = os.path.join(test_folder_path, image_file)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        plt.imshow (img, cmap = 'gray' )
        plt.show ()
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        img = img/255
        input_img = np.expand_dims(img, axis=0)
        Predictions = model.predict(input_img)

        predicted_class = CATEGORIES[np.argmax(Predictions)]
        confidence = np.max(Predictions)


        if  confidence >= 0.8:
            print (predicted_class, confidence)
        else:
            print ("unrecognisable class")
else:
    print(f"The folder '{folder_name}' doesn't exist")

#Converting to tflite
Tensorflow lite allows models to run on resource constrained devices such raspberry pi and other microcontrollers. The conversion of a keras model is document on [Tensorflow's docs](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter).

In [ ]:
import tensorflow as tf

DATADIR = data_path
CATEGORIES = ['person', 'car']
IMG_SIZE = 50

def representative_data_gen():
    data_dir = DATADIR
    batch_size = 32
    img_height = IMG_SIZE
    img_width = IMG_SIZE

    class_names = CATEGORIES

    train_images = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        class_names=class_names,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)

    #standardize the images
    normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1/255)
    normalized_ds = train_images.map(lambda x, y: (normalization_layer(x), y))
    image_batch, labels_batch = next((iter(normalized_ds)))
    first_image = image_batch[0]
    #print(image_batch)

    print(np.min(first_image), np.max(first_image))

    for input_value in tf.data.Dataset.from_tensor_slices(image_batch).batch(1).take(100):
        print(input_value.shape)
        # Model has only one input so each data point has one element.
        yield [tf.constant(input_value, dtype=tf.float32, shape=(1, img_height, img_width, 1))]

In [ ]:
representative_data_gen()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model = converter.convert()

In [ ]:
with open(save_path_lite, "wb") as f:
    f.write(tflite_quant_model)

In [ ]:
interpreter = tf.lite.Interpreter(save_path_lite)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)

In [ ]:
summary = interpreter.get_signature_list()
print("Model Summary:", summary)

In [ ]:
input_tensor_shape = input_details[0]['shape']
output_tensor_shape = output_details[0]['shape']
input_tensor_type = input_details[0]['dtype']
output_tensor_type = output_details[0]['dtype']

print("Input Tensor Shape:", input_tensor_shape)
print("Output Tensor Shape:", output_tensor_shape)
print("Input Tensor Type:", input_tensor_type)
print("Output Tensor Type:", output_tensor_type)

#Model deployment
As deployment will be on resource constrained devices, such as esp32, [edge impulse](https://docs.edgeimpulse.com/docs/edge-impulse-studio/deployment) will be used to deploy the model to an Arduino-compatible library that will run on the microcontroller.

In [ ]:
!pip install edgeimpulse

In [ ]:
import edgeimpulse as ei
from google.colab import userdata

API_KEY = userdata.get('EI_API_KEY')

ei.API_KEY = API_KEY

In [ ]:
ei.model.list_profile_devices()

In [ ]:
profile = ei.model.profile(model=tflite_model, device='espressif-esp32')
print(profile.summary())

In [ ]:
print(f"Estimated RAM usage: {profile.model.profile_info.float32.memory.tflite.ram}")
print(f"Estimated ROM usage: {profile.model.profile_info.float32.memory.tflite.rom}")
print(f"Estimated inference time (ms): {profile.model.profile_info.float32.time_per_inference_ms}")

In [ ]:
ei.model.list_deployment_targets()

In [ ]:
ei.model.deploy(model=tflite_model,
                model_input_type=ei.model.input_type.OtherInput(),
                model_output_type=ei.model.output_type.Classification(),
                output_directory=Lib_folder)